In [2]:
import os
from PIL import Image
from pathlib import Path
from dotenv import load_dotenv
import dataset
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

load_dotenv()

data_path = Path(os.getenv("prepare_dataset_folder"))
processed_path = Path(os.getenv("processed_images_folder"))
split = "train"  # Adjust as needed for 'val' or 'test'

# Ensure the processed data folder exists
processed_path.mkdir(parents=True, exist_ok=True)

# Instantiate the dataset
forest_sat_dataset = dataset.MineSATDataset(split=split, data_path=data_path)

# Iterate over all dataset indices
for index in range(len(forest_sat_dataset)):
    # Get the filepaths for the current index
    filepath = forest_sat_dataset.filepaths[index]

    # Extract the original directory name
    original_dir_name = Path(filepath).name

    # Get the transformed images
    images_dict = forest_sat_dataset.get_images(
        index)  # Ensure you call the correct method

    # Create a new directory path for the processed images
    processed_dir_path = processed_path / original_dir_name
    processed_dir_path.mkdir(parents=True, exist_ok=True)

    for img_type, image in images_dict.items():
        # Skip saving the mask if you only want the indices and RGB images
        if img_type == "Mask":
            continue

        if img_type == "NBR":
            # Apply color mapping for NBR
            plt.imshow(image, cmap=plt.cm.RdYlGn, vmin=-1, vmax=1)
            plt.colorbar(orientation='vertical')
            plt.axis('off')

            # Save the current figure to a numpy array
            fig = plt.gcf()
            plt.draw()
            image_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            image_np = image_np.reshape(
                fig.canvas.get_width_height()[::-1] + (3,))

            # Convert numpy array to PIL Image
            pil_image = Image.fromarray(image_np)

            # Define the filename, replacing .tif with .png for NBR
            filename = f"{img_type}.png"

            # Close the figure to free memory
            plt.close(fig)
        else:
            # Normalize the image data to 0-255 for other image types
            # Clip to the range you want
            image = np.clip(image, 0, np.max(image))
            image_8bit = ((image - np.min(image)) /
                          (np.max(image) - np.min(image)) * 255).astype('uint8')

            # If the image has more than one channel, convert it to RGB
            if image_8bit.ndim > 2 and image_8bit.shape[2] > 3:
                # Convert multi-band images (e.g., 4 bands) to RGB (3 bands) before saving as JPEG
                image_8bit = image_8bit[:, :, :3]

            # Create the PIL Image from the numpy array
            pil_image = Image.fromarray(image_8bit)

            # Define the filename, replacing .tif with .jpg for other image types
            filename = f"{img_type}.jpg"

        # Define the full path for the file
        filepath = processed_dir_path / filename

        # Save the image
        pil_image.save(filepath)


/var/folders/qp/39q3q5193vb9k27hwzyk9fk80000gn/T/ipykernel_17015/2599301494.py:52: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/Users/annabelle/Documents/NEU/CS7980/Capstone/mine_seg_sat/dataset.py:250: RuntimeWarning: invalid value encountered in divide
  return band / np.percentile(band, percentile)
/var/folders/qp/39q3q5193vb9k27hwzyk9fk80000gn/T/ipykernel_17015/2599301494.py:68: RuntimeWarning: invalid value encountered in divide
  image_8bit = ((image - np.min(image)) /
/var/folders/qp/39q3q5193vb9k27hwzyk9fk80000gn/T/ipykernel_17015/2599301494.py:68: RuntimeWarning: invalid value encountered in cast
  image_8bit = ((image - np.min(image)) /
/Users/annabelle/Documents/NEU/CS7980/Capstone/mine_seg_sat/dataset.py:250: RuntimeWarning: divide by zero encountered in divide
  return band / np.percentile(